In [1]:
import requests as rq
import pandas as pd

# defining the url for exploring the spacex api
url = 'https://api.spacexdata.com/v4/launches'

# making a GET request
response = rq.get(url)

# to see whether the request succeeds
if response.status_code == 200:
    data = response.json() # analysis of the json data
    df = pd.DataFrame.from_dict(data) # converting python dictionary into pandas (pd) dataframe
    print(df.head())
else:
    print('Failed to fetch data from the API') # when request (rq) instead fails

                                            fairings  \
0  {'reused': False, 'recovery_attempt': False, '...   
1  {'reused': False, 'recovery_attempt': False, '...   
2  {'reused': False, 'recovery_attempt': False, '...   
3  {'reused': False, 'recovery_attempt': False, '...   
4  {'reused': False, 'recovery_attempt': False, '...   

                                               links  \
0  {'patch': {'small': 'https://images2.imgbox.co...   
1  {'patch': {'small': 'https://images2.imgbox.co...   
2  {'patch': {'small': 'https://images2.imgbox.co...   
3  {'patch': {'small': 'https://images2.imgbox.co...   
4  {'patch': {'small': 'https://images2.imgbox.co...   

       static_fire_date_utc  static_fire_date_unix    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False     0.0   
1                      None                    NaN  False     0.0   
2                      None                    NaN  False     0.0   
3  2008-09-20T00:00:00.000Z           1.221869e+09

In [2]:
# repeating the same for exploring another endpoint
launchpads_url = 'https://api.spacexdata.com/v4/launchpads'
launchpads_response = rq.get(url)

if launchpads_response.status_code == 200: # 200 means OK while 404 or 500 would mean that the data wasn't retrieved properly: Server Error
    launchpads_data = launchpads_response.json()
    df_launchpads = pd.DataFrame.from_dict(launchpads_data)
    print(df_launchpads.tail())
else:
    print('Failed to fetch data from the API... Try again!')
    
# launches and launchpads have differences in what is included in their columns and rows, especially since i deliberately chose to use the head function on launches and the tail function on launchpads to extract more differences in their insights

                                              fairings  \
200  {'reused': None, 'recovery_attempt': None, 're...   
201  {'reused': None, 'recovery_attempt': None, 're...   
202  {'reused': None, 'recovery_attempt': None, 're...   
203  {'reused': None, 'recovery_attempt': None, 're...   
204  {'reused': None, 'recovery_attempt': None, 're...   

                                                 links static_fire_date_utc  \
200  {'patch': {'small': None, 'large': None}, 'red...                 None   
201  {'patch': {'small': None, 'large': None}, 'red...                 None   
202  {'patch': {'small': None, 'large': None}, 'red...                 None   
203  {'patch': {'small': None, 'large': None}, 'red...                 None   
204  {'patch': {'small': None, 'large': None}, 'red...                 None   

     static_fire_date_unix    net  window                    rocket success  \
200                    NaN  False     NaN  5e9d0d95eda69973a809d1ec    None   
201               

In [3]:
df_launchpads.describe()

,static_fire_date_unix,window,flight_number,date_unix
count,1.210000e+02,117.000000,205.000000,2.050000e+02
mean,1.520206e+09,2568.974359,102.814634,1.566077e+09
std,9.080036e+07,4389.948430,59.029112,1.037329e+08
min,1.142554e+09,0.000000,1.000000,1.143239e+09
25%,1.475417e+09,0.000000,52.000000,1.513992e+09
50%,1.529789e+09,0.000000,103.000000,1.601984e+09
75%,1.589368e+09,5280.000000,154.000000,1.647660e+09
max,1.650464e+09,18300.000000,203.000000,1.670198e+09


In [4]:
df.dtypes

fairings                  object
links                     object
static_fire_date_utc      object
static_fire_date_unix    float64
net                         bool
window                   float64
rocket                    object
success                   object
failures                  object
details                   object
crew                      object
ships                     object
capsules                  object
payloads                  object
launchpad                 object
flight_number              int64
name                      object
date_utc                  object
date_unix                  int64
date_local                object
date_precision            object
upcoming                    bool
cores                     object
auto_update                 bool
tbd                         bool
launch_library_id         object
id                        object
dtype: object

In [5]:
df_launchpads.dtypes # same types of 27 columns for both launches and launchpads

fairings                  object
links                     object
static_fire_date_utc      object
static_fire_date_unix    float64
net                         bool
window                   float64
rocket                    object
success                   object
failures                  object
details                   object
crew                      object
ships                     object
capsules                  object
payloads                  object
launchpad                 object
flight_number              int64
name                      object
date_utc                  object
date_unix                  int64
date_local                object
date_precision            object
upcoming                    bool
cores                     object
auto_update                 bool
tbd                         bool
launch_library_id         object
id                        object
dtype: object

In [6]:
df.describe() # launchpads statistics had more meaningful insights than what we get below for launches

,static_fire_date_unix,window,flight_number,date_unix
count,1.210000e+02,117.000000,205.000000,2.050000e+02
mean,1.520206e+09,2568.974359,102.814634,1.566077e+09
std,9.080036e+07,4389.948430,59.029112,1.037329e+08
min,1.142554e+09,0.000000,1.000000,1.143239e+09
25%,1.475417e+09,0.000000,52.000000,1.513992e+09
50%,1.529789e+09,0.000000,103.000000,1.601984e+09
75%,1.589368e+09,5280.000000,154.000000,1.647660e+09
max,1.650464e+09,18300.000000,203.000000,1.670198e+09


In [7]:
# using a variety of routes and calls to get a handle for making multiple API requests
rockets_url = 'https://api.spacexdata.com/v4/rockets'
all_rockets = []
limit = 10
page = 1

# using a while loop where the response has the specified limit and starting point hardcoded within the link
while True:
    try:
        rockets_response = rq.get(f'{rockets_url}?limit={limit}&page={page}', timeout=10)
    
        # the usual check-in for the successful request
        if rockets_response.status_code != 200:
            print('Failed to fetch data from the API')
            break # exit if an error occurs
    
        rockets_data = rockets_response.json()
        
        if not rockets_data:  # no data retrieved means we need to end pagination by breaking the loop
            print('No more data to fetch')
            break
    
        all_rockets.extend(rockets_data)  # extend list from current page's data
        page+=1  # on to next page
    
    except rq.exceptions.RequestException as e:
        print(f'An error occurred: {e}')
        break
        
# calling the usual same function
df_rockets = pd.DataFrame(all_rockets)
df_rockets.head()

An error occurred: HTTPSConnectionPool(host='api.spacexdata.com', port=443): Read timed out. (read timeout=10)


,height,diameter,mass,first_stage,second_stage,engines,landing_legs,payload_weights,flickr_images,name,...,stages,boosters,cost_per_launch,success_rate_pct,first_flight,country,company,wikipedia,description,id
0,"{'meters': 22.25, 'feet': 73}","{'meters': 1.68, 'feet': 5.5}","{'kg': 30146, 'lb': 66460}","{'thrust_sea_level': {'kN': 420, 'lbf': 94000}...","{'thrust': {'kN': 31, 'lbf': 7000}, 'payloads'...","{'isp': {'sea_level': 267, 'vacuum': 304}, 'th...","{'number': 0, 'material': None}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...","[https://imgur.com/DaCfMsj.jpg, https://imgur....",Falcon 1,...,2,0,6700000,40,2006-03-24,Republic of the Marshall Islands,SpaceX,https://en.wikipedia.org/wiki/Falcon_1,The Falcon 1 was an expendable launch system p...,5e9d0d95eda69955f709d1eb
1,"{'meters': 70, 'feet': 229.6}","{'meters': 3.7, 'feet': 12}","{'kg': 549054, 'lb': 1207920}","{'thrust_sea_level': {'kN': 7607, 'lbf': 17100...","{'thrust': {'kN': 934, 'lbf': 210000}, 'payloa...","{'isp': {'sea_level': 288, 'vacuum': 312}, 'th...","{'number': 4, 'material': 'carbon fiber'}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...",[https://farm1.staticflickr.com/929/2878733830...,Falcon 9,...,2,0,50000000,98,2010-06-04,United States,SpaceX,https://en.wikipedia.org/wiki/Falcon_9,Falcon 9 is a two-stage rocket designed and ma...,5e9d0d95eda69973a809d1ec
2,"{'meters': 70, 'feet': 229.6}","{'meters': 12.2, 'feet': 39.9}","{'kg': 1420788, 'lb': 3125735}","{'thrust_sea_level': {'kN': 22819, 'lbf': 5130...","{'thrust': {'kN': 934, 'lbf': 210000}, 'payloa...","{'isp': {'sea_level': 288, 'vacuum': 312}, 'th...","{'number': 12, 'material': 'carbon fiber'}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...",[https://farm5.staticflickr.com/4599/385838292...,Falcon Heavy,...,2,2,90000000,100,2018-02-06,United States,SpaceX,https://en.wikipedia.org/wiki/Falcon_Heavy,With the ability to lift into orbit over 54 me...,5e9d0d95eda69974db09d1ed
3,"{'meters': 118, 'feet': 387}","{'meters': 9, 'feet': 30}","{'kg': 1335000, 'lb': 2943000}","{'thrust_sea_level': {'kN': 128000, 'lbf': 287...","{'thrust': {'kN': 1957, 'lbf': 440000}, 'paylo...","{'isp': {'sea_level': 330, 'vacuum': 380}, 'th...","{'number': 6, 'material': 'stainless steel'}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...",[https://live.staticflickr.com/65535/489541389...,Starship,...,2,0,7000000,0,2021-12-01,United States,SpaceX,https://en.wikipedia.org/wiki/SpaceX_Starship,Starship and Super Heavy Rocket represent a fu...,5e9d0d96eda699382d09d1ee
4,"{'meters': 22.25, 'feet': 73}","{'meters': 1.68, 'feet': 5.5}","{'kg': 30146, 'lb': 66460}","{'thrust_sea_level': {'kN': 420, 'lbf': 94000}...","{'thrust': {'kN': 31, 'lbf': 7000}, 'payloads'...","{'isp': {'sea_level': 267, 'vacuum': 304}, 'th...","{'number': 0, 'material': None}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...","[https://imgur.com/DaCfMsj.jpg, https://imgur....",Falcon 1,...,2,0,6700000,40,2006-03-24,Republic of the Marshall Islands,SpaceX,https://en.wikipedia.org/wiki/Falcon_1,The Falcon 1 was an expendable launch system p...,5e9d0d95eda69955f709d1eb


In [9]:
# after pagination, finally checking the number of rows and columns (more as a confirmation of the pagination process)
df_rockets.shape # 185,672 rows and 22 columns!

(185672, 22)